In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb

import os
from pathlib import Path

# Styling
color_pal = sns.color_palette("Set2")
plt.style.use('seaborn-v0_8-colorblind') 

# Warnings control
import warnings 
warnings.filterwarnings('ignore')

In [2]:
data_path = Path('stock-price-predictions/price/AAPL.csv')

df = pd.read_csv(data_path, header=0)

In [3]:
df.set_index('Date', inplace=True)
df.index = pd.to_datetime(df.index)

train = df.loc[df.index < '01-01-2016']
test = df.loc[df.index >= '01-01-2016']